# Poisson likelihood calculator

Given $M$ components, each with an estimated rate $\vec{\beta}$ determined by a normal distribution with uncertainty $\vec{\sigma}$, calculate the confidence itervals and perform a hypothesis tests for each parameter $b$.

Nominally each event corresponds to a set of observables $\vec{x}$ of $N$ measurements, for any given measurement, the probability for that particular measurement to come from a particular components is given by

$$ P_i(\vec{x}) $$

The prior probability is then formed through a combination of these components such that the total probability is 

$$ \mathbf{P} = \sum_i^M P_i(\vec{x}) $$

The likelihood for a full data set of $N$ measurements is the product of each event total probability

$$\mathcal{L}(\vec{x}) = \prod_j^N \left( \sum_i^M b_iP_i(\vec{x}) \right) / \sum_i^Mb_i $$

We can extend the likelihood by proclaiming that each components as well as the sum of components are simply a stochastic process, produces the extended likelihood:

$$\mathcal{L}(\vec{x}) = \frac{\text{e}^{-\sum_i^Mb_i}}{N!} \prod_j^N \left( \sum_i^M b_iP_i(\vec{x}) \right) $$

Finally, we can claim that we have _a priori_ knowledge of the parameters, whether it be through side-band analysis or external constraints, by including those constraints via some prior probability. Given no specific knowledge of the shape of that prior, we will consider the information we receive on the variables to be normally distributed and multiply the likelihood by those constraints

$$\mathcal{L}(\vec{x}) = \frac{\text{e}^{-\sum_i^Mb_i}}{N!} \prod_j^N \left( \sum_i^M b_iP_i(\vec{x}) \right) \frac{1}{\sqrt{2\pi \sigma_j^2}}\text{exp}\left({\frac{-(\beta_i-b_i)^2}{2\sigma_i^2}}\right)$$

A few definitions to simplify things:
$$ \lambda := \sum_i^Mb_i $$

Then then our objective function $\mathcal{O} = -\text{Ln}\mathcal{L}$

$$\mathcal{O} = \lambda + \text{Ln}N! -\sum_j^N\text{Ln}\left( \sum_i^M b_iP_i(\vec{x}) \right) + \sum_i^M \left( \frac{(\beta_i-b_i)^2}{2\sigma_i^2} + \text{Ln}\sqrt{2\pi \sigma_i} \right)$$

Finally, for a counting analysis we assume that an optimal set of cuts has been applied which optimizes the sensitivity to a particular parameter, which simplifies the likelihood such that
$$ P_i(\vec{x}) := 1 $$
Also, because the shape of the likelihood space is independent of constant parameters, we can drop the $\text{Ln}\sqrt{2\pi \sigma_i}$ terms. We could also remove the $\text{Ln}N!$ term as well, but for numerical stability we will keep it around, but use Sterling's approximation: $\text{Ln}N! \approx N\text{Ln}N - N$. The remaining objective function we will thus use is:

$$\mathcal{O} = \lambda - N\text{Ln}\lambda + N\text{Ln}N - N + \sum_i^M \left( \frac{(\beta_i-b_i)^2}{2\sigma_i^2} \right)$$

_Note: If the different values of $\beta$ differ by orders of magnitude, it might be worth forming an affine invariant form of the likelihood, otherwise the $\text{Ln}\sqrt{2\pi \sigma_i}$ term should not matter_

In [1]:
include("./watchfish.jl")
using .watchfish

In [2]:
m = Model()
add_component!(m, "Signal", 20.0; σ=Inf)
add_component!(m, "Thing",  30.0; σ=10.0)
add_component!(m, "Thing2", 40.0; σ=30.0)
add_component!(m, "Thing3", 10.0; σ=20.0)
add_component!(m, "Thing4",  10.0; σ=10.0)
add_component!(m, "Thing5", 20.0; σ=30.0)
add_component!(m, "Thing6", 10.0; σ=20.0)

data = 230 #events
results = run_fish(m, data)

@show results

Got 9.329882459084541e-13 at [10.000005115626163, 30.000002541040715, 10.000003649055872, 40.00002676166872, 20.00002282489499, 109.99993048690655, 9.999999424326171] after 2938 iterations
results = Results(Opt(LN_SBPLX, 7), Model(Dict("Thing3" => Component("Thing3", 10.0, 20.0),"Thing" => Component("Thing", 30.0, 10.0),"Thing4" => Component("Thing4", 10.0, 10.0),"Thing2" => Component("Thing2", 40.0, 30.0),"Thing5" => Component("Thing5", 20.0, 30.0),"Signal" => Component("Signal", 20.0, Inf),"Thing6" => Component("Thing6", 10.0, 20.0)), 7), 9.329882459084541e-13, [10.000005115626163, 30.000002541040715, 10.000003649055872, 40.00002676166872, 20.00002282489499, 109.99993048690655, 9.999999424326171], 2938, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf, Inf, Inf])


Results(Opt(LN_SBPLX, 7), Model(Dict("Thing3" => Component("Thing3", 10.0, 20.0),"Thing" => Component("Thing", 30.0, 10.0),"Thing4" => Component("Thing4", 10.0, 10.0),"Thing2" => Component("Thing2", 40.0, 30.0),"Thing5" => Component("Thing5", 20.0, 30.0),"Signal" => Component("Signal", 20.0, Inf),"Thing6" => Component("Thing6", 10.0, 20.0)), 7), 9.329882459084541e-13, [10.000005115626163, 30.000002541040715, 10.000003649055872, 40.00002676166872, 20.00002282489499, 109.99993048690655, 9.999999424326171], 2938, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [Inf, Inf, Inf, Inf, Inf, Inf, Inf])

In [3]:
function Base.show(io::IO, m::Results)
    compact = get(io, :compact, false)

    if !compact
        println("Fit converged after $(m.iterations) iterations")
        println("Best fit at: -LnL = $(m.min_objective)")
        println("Values of $(m.min_parameters)")
    end
        #show(io, m.iterations)
end

@show results

Fit converged after 2938 iterations
Best fit at: -LnL = 9.329882459084541e-13
Values of [10.000005115626163, 30.000002541040715, 10.000003649055872, 40.00002676166872, 20.00002282489499, 109.99993048690655, 9.999999424326171]
results = 


Fit converged after 2938 iterations
Best fit at: -LnL = 9.329882459084541e-13
Values of [10.000005115626163, 30.000002541040715, 10.000003649055872, 40.00002676166872, 20.00002282489499, 109.99993048690655, 9.999999424326171]


In [ ]:
## Setup the tools
using NLopt

## Model variables
M = 4
β = (20.0, 30.0, 40.0, 10.0)
σ = (Inf, 10.0, 30.0, 20.0) # Inf means unconstrained
p0 = [10.0, 10.0, 10.0, 10.0]
## Measurements
N = 160

function constraint(x, μ, σ)
    if σ == Inf
        return 0
    end
    if σ ≈ 0
        return Inf
    end
    (x-μ)^2/2/σ^2
end

function objective(x::Vector, grad::Vector)
    if length(grad)>0
        grad = 2x
    end
    λ = sum(x)
    ξ = sum( constraint.(x, β, σ) )
    λ - N*log(λ) + N*log(N) - N + ξ
end

## Setup the optimizer, here we use nlopt with the sublex method
## which works well for higher dimensionality on problems without
## a known gradient.
opt = Opt(:LN_SBPLX, M)
opt.ftol_rel = 1e-4
opt.min_objective = objective
# Initial guess -- just need to be close-ish

## Test constraint, move below
#opt.lower_bounds = [1.0, -Inf, -Inf]
#opt.upper_bounds = [1.0, Inf, Inf]
## end test
(minf, minx, ret) = optimize!(opt, p0)

numevals = opt.numevals
println("Got $minf at $minx after $numevals iterations (returned $ret)")

low  = copy(opt.lower_bounds)
high = copy(opt.upper_bounds)
best = copy(minx)
minl = copy(minf)

@show opt

In [ ]:
## Profile likelihood of each parameter
## WITHOUT a prior, we can take the normalized objective function
using PyPlot
using NLopt

## Everything stored in results
M = results.model.dims
fig, ax = subplots(nrows=M, ncols=M)

## We will be lazy for now and robust later. Lazy way, assume we know the range
x = collect(0:1:100)
y = collect(0:1:100)
X = repeat( reshape(x, 1, :), length(y), 1)
Y = repeat(y, 1, length(x))
# What does a slice in space look like?
# We can optimize again, with a constraint in place (bounds)
 
for i in collect(1:M)
    for j in collect(i:M)
        # profile this parameter
        nlow = copy(results.lower_bounds)
        nhigh = copy(results.upper_bounds)
        p0 = copy(results.min_parameters)
        nll = []
        if i != j
            for a in x
                for b in y
                    nlow[i], nlow[j] = a, b
                    nhigh[i], nhigh[j] = a, b
                    results.opt.lower_bounds = nlow
                    results.opt.upper_bounds = nhigh
                    #p0 = copy(results.min_parameters)
                    p0[i], p0[j] = a, b
                    minf, minx, ret = optimize!(results.opt, p0)
                    push!(nll, exp(-minf) )
                end
            end
            Z = reshape(nll, length(x), length(y))
            ax[j,i].contourf(x, y, Z)
            continue
        end
        for a in x
            nlow[i] = a
            nhigh[i] = a
            results.opt.lower_bounds = nlow
            results.opt.upper_bounds = nhigh
            p0[i] = a
            #@show opt.lower_bounds[i]
            minf, minx, ret = optimize!(results.opt, p0)
            #@show ret minf p0
            push!(nll, exp(-minf) )
        end
        ax[i,i].plot(x, nll)
        ax[i,i].set_ylim(0, 1)
    end
end
## Lets do a diagonal plot
show()

In [ ]:
using NLopt
using PyPlot
function profileSingle(idx, results; stop=6, step=1.0)
    # Go left
    x_left = []
    nll_left = []
    nlow = copy(results.lower_bounds)
    nhigh = copy(results.upper_bounds)
    p0 = copy(results.min_parameters)
    val = 0
    xeval = p0[idx]
    while (val-results.min_objective) < stop
        nlow[idx] = xeval
        nhigh[idx] = xeval
        results.opt.lower_bounds = nlow
        results.opt.upper_bounds = nhigh
        p0[idx] = xeval
        val, minx, ret = optimize!(results.opt, p0)
        push!(nll_left, exp(-(val-results.min_objective)) )
        push!(x_left, xeval)
        xeval -= step
    end
    # Go right
    x_right = []
    nll_right = []
    nlow = copy(results.lower_bounds)
    nhigh = copy(results.upper_bounds)
    p0 = copy(results.min_parameters)
    val = 0
    xeval = p0[idx]
    while (val-results.min_objective) < stop
        nlow[idx] = xeval
        nhigh[idx] = xeval
        results.opt.lower_bounds = nlow
        results.opt.upper_bounds = nhigh
        p0[idx] = xeval
        val, minx, ret = optimize!(results.opt, p0)
        push!(nll_right, exp(-(val-results.min_objective)) )
        push!(x_right, xeval)
        xeval += step
    end
    x = append!(reverse(x_left), x_right)
    nll = append!(reverse(nll_left), nll_right)
    return x, nll
end

function uncertainty(likelihood_x, likelihood_y, α ; mode="FC")
    # Mode can be FC (Feldman-Cousins), Mode-centered, mean-centered, left, right, etc.
    # Return cumulative, x_low, x_high
    likelihood_y = likelihood_y / sum(likelihood_y)
    cumulative = cumsum(likelihood_y)
    β = 1 - α
    @show β
    if mode == "FC"
        cy = sortperm(likelihood_y; rev=true)
        cum_y = cumsum(likelihood_y[cy])
        cum_x = likelihood_x[cy]
        region = cum_x[ cum_y .<= α ]
        left = minimum(region)
        right = maximum(region)
    end
    left, right
end

x, y = profileSingle(4, results; stop=6, step=1)
l, r = uncertainty(x, y, 0.90)

@show l r
plot(x, y)
axvline(l, color="red")
axvline(r, color="red")
ylim(bottom=0)
show()

In [ ]:
function profileSingle(idx, opt, bestfit, minfit; stop=6, step=1.0)
    # Go left
    x_left = []
    nll_left = []
    nlow = copy(opt.lower_bounds)
    nhigh = copy(opt.upper_bounds)
    p0 = copy(bestfit)
    val = 0
    xeval = p0[idx]
    while (val-minfit) < stop
        nlow[idx] = xeval
        nhigh[idx] = xeval
        opt.lower_bounds = nlow
        opt.upper_bounds = nhigh
        p0[idx] = xeval
        val, minx, ret = optimize!(opt, p0)
        push!(nll_left, exp(-(val-minfit)) )
        push!(x_left, xeval)
        xeval -= step
    end
    # Go right
    x_right = []
    nll_right = []
    nlow = copy(opt.lower_bounds)
    nhigh = copy(opt.upper_bounds)
    p0 = copy(bestfit)
    val = 0
    xeval = p0[idx]
    while (val-minfit) < stop
        nlow[idx] = xeval
        nhigh[idx] = xeval
        opt.lower_bounds = nlow
        opt.upper_bounds = nhigh
        p0[idx] = xeval
        val, minx, ret = optimize!(opt, p0)
        push!(nll_right, exp(-(val-minfit)) )
        push!(x_right, xeval)
        xeval += step
    end
    x = append!(reverse(x_left), x_right)
    nll = append!(reverse(nll_left), nll_right)
    return x, nll
end

function uncertainty(likelihood_x, likelihood_y, α ; mode="FC")
    # Mode can be FC (Feldman-Cousins), Mode-centered, mean-centered, left, right, etc.
    # Return cumulative, x_low, x_high
    likelihood_y = likelihood_y / sum(likelihood_y)
    cumulative = cumsum(likelihood_y)
    β = 1 - α
    @show β
    if mode == "FC"
        cy = sortperm(likelihood_y; rev=true)
        cum_y = cumsum(likelihood_y[cy])
        cum_x = likelihood_x[cy]
        region = cum_x[ cum_y .<= α ]
        left = minimum(region)
        right = maximum(region)
    end
    left, right
end

x, y = profileSingle(1, opt, best, minl; stop=6, step=1)
l, r = uncertainty(x, y, 0.90)

@show l r
plot(x, y)
axvline(l, color="red")
axvline(r, color="red")
ylim(bottom=0)
show()

In [ ]:
## Testing bits, write and move
mutable struct Component
    name::String
    μ::Float64
    σ::Float64
end

mutable struct Model
    component_dict::Dict{String, Component}
    ndims::Int64
    function Model()
        new(Dict{String, Component}(), 0)
    end
end

function show(m::Model)
    m.component_dict
end

function add_component!(m::Model, name::String, μ; σ=Inf)
    get!(m.component_dict, name, Component(name, μ, σ))
    m.ndims = length(m.component_dict)
end
    
m = Model()
add_component!(m, "Signal", 1.7; σ=Inf)

@show m